In [12]:
import os
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize
import timm
import numpy as np
import albumentations as A
from albumentations.pytorch import ToTensorV2
import PIL

In [13]:
# 테스트 데이터셋 폴더 경로를 지정해주세요.
test_dir = '/home/hj/Study/MLops/Test_Data|'

In [14]:
class CNN(nn.Module):
    """
    Base Implement Model to training
    """
    def __init__(self, class_n, rate=0.1):
        super(CNN, self).__init__()
        self.model = models.resnet50(pretrained=True)
        self.classifier1 = nn.Linear(1000,25)
        self.dropout = nn.Dropout(rate)
    
    def forward(self, inputs):
        output = self.model(inputs)
        output = self.dropout(self.classifier1(output))# dropout 조심.
        return output

In [15]:
A_transform = {
    "train_Filp_transform": A.Compose(
        [
            A.OneOf(
                [
                    A.HorizontalFlip(p=1.0),
                    A.VerticalFlip(p=1.0),
                    A.RandomRotate90(p=1.0),
                ],
                p=0.75,
            ),
            A.RandomBrightness(p=0.5, limit=(-0.2, 0.25)),
            A.RandomContrast(p=0.5, limit=(-0.25, 0.25)),
            ToTensorV2(p=1.0),  # dtype float32, transpose 한번에
        ]
    ),
    "train_deault_transform": A.Compose([ToTensorV2(p=1.0)]),
}


In [16]:

class BaseDataset(Dataset):
    def __init__(self, dataset_df, transform=None):
        super().__init__()
        assert transform is not None, "Set the transform on train set"
        self.transform = transform
        self.dataset_df = dataset_df

    def __len__(self):
        return len(self.dataset_df)

    def __getitem__(self, idx):
        # TODO : Image Path를 csv 파일에 추가하는게 좋은 선택지 같음.
        self.dataset_df['image_path']
        image = cv2.imread(self.dataset_df['image_path'].iloc[idx])
        # TODO : TOTensorV2를 쓰면 numpy 채널 축도 변하고 ToTensor의 기능을 포함함.
        # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = self.transform(image=image)['image'].float()
        return image, torch.tensor(self.dataset_df['label'].iloc[idx])
    
    def get_labels(self):
        return self.dataset_df['label'].iloc[:]

class CropDataset():
    def __init__(self, transform=None, default_transform=None, config=None):
        super().__init__()
        # TODO : 다양한 assert문 utils로 추가해놓기 (params : variable name)
        assert transform is not None, "Set the transform on train set"
        assert default_transform is not None, "Set the default transform"

        self.transform = transform
        self.default_transform = default_transform
        self.dir_path = os.path.dirname(config['Prepared_Data_Path'])
        self.df_path = os.path.join(self.dir_path, 'train.csv')
        self.img_dir_path = config['Prepared_Data_Path']

        self.dataset_df = pd.read_csv(os.path.join(config['Prepared_Csv_Path'], 'train.csv'))
        # TODO : Class Imbalance 문제를 해결하기 위해 각 class weight를 지정하는 코드 Cross_Entropy를 사용할 때 사용할 듯
        # self.class_weight = self._get_class_weight() # 학습 고도화를 위한 각 class Imbalance 문제를 위한 코드


    def split_validation(self, valid_split_ratio):
        # TODO : Stratify의 처리가 제대로되는지 확인하기
        df_train, df_val = train_test_split(self.dataset_df, test_size=valid_split_ratio, random_state=42, stratify=self.dataset_df['label'].to_numpy())
        train_dataset = BaseDataset(df_train, transform=self.transform)
        val_dataset = BaseDataset(df_val, transform=self.default_transform)
        return train_dataset, val_dataset

In [ ]:
from tqdm import tqdm
from glob import glob

image_dir = sorted(glob(os.path.join(test_dir, '*/*.jpg')))
train_dataset, val_dataset  = CropDataset()